# Lab 1 

Laboration 1 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-1/

### The objectives of this assignment are to:

-Write a program that collects all the words from a set of documents

-Build an index from the words

-Know what indexing is

-Represent a document using the Tf.Idf value

-Write a short report of 1 to 2 pages on the assignment

-Read a short text on an industrial system

## Indexing one file

* The index file will contain all the unique words in the document, where each word is associated with the list of its positions in the document.

* You will represent this index as a dictionary where the keys will be the words and the values, the lists of positions

* As words, you will consider all the strings of letters that you will set in lower case. You will not index the rest (i.e. numbers or symbols).

This is done by first using function txtClean.

In [107]:
import re

def txtClean(text):
    """
    Replace capital characters to small characters
    
    Input txt file
    Output txt file
    """
    # Remove new lines
    text = re.sub("\n", " ", text)
    
    # Replace [A-Ö] with [a-ö]
    def toLowercase(matchobj):
        return matchobj.group(1).lower()
    
    text = re.sub(r'([A-Z])', toLowercase, text)
    
    # Remove multiple spaces
    text = re.sub(' +', ' ', text)

    return text    

In [108]:
text = open("Selma/bannlyst.txt").read()  
txt = txtClean(text)

* To extract the words, you will use Unicode regular expressions. Do not use \w+, for instance, but the Unicode equivalent. The word positions will correspond to the number of characters from the beginning of the file. (The word offset from the beginning)

* You will use finditer() to find the positions of the words. This will return you match objects, where you will get the matches and the positions with the group() and start() methods.

In [182]:
def string2dict(text,originaltext):
    """
    Creates a dict with (word:list[index apperences]) from input string
    
    Input string, string
    Output dict
    """
    stringList = re.findall(r"[a-zåäö]+",text) #This finds all words from a txt file. r"[a-zåäö]+ equal to r"\w+" 

    stringDict = dict.fromkeys(stringList) #Creates dict (and remove doublicates)

    for word in stringDict:
        wordIndices = []
        pattern = r"\b"+word+ r"\b" #Only look at word 
        
        for m in re.finditer(pattern, originaltext): #Iterate thorugh every word
            wordIndices.append(m.start(0))
            
        stringDict.update({word:wordIndices})
    return stringDict

In [183]:
txtDict = string2dict(txt,text) 

In [214]:
len(txtDict)

7950

Test with bannlyst text. The word gjord occurs three times in the text at positions 8551, 183692, and 220875, uppklarnande, once at position 8567, and stjärnor, once at position 8590. 

In [185]:
txtDict['gjord']

[8551, 183692, 220875]

In [188]:
txtDict['uppklarnande']

[8567]

In [186]:
txtDict['stjärnor']

[8590]

* You will use the pickle package to write your dictionary in an file, see https://wiki.python.org/moin/UsingPickle.

In [189]:
import pickle

with open('BannlystTxtDict.pickle', 'wb') as handle:
    pickle.dump(txtDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Open pickle

In [190]:
with open('BannlystTxtDict.pickle', 'rb') as handle:
    BannlystTxtDict = pickle.load(handle)

Test

In [192]:
BannlystTxtDict == txtDict

True

## Reading the content of a folder

Write a function that reads all the files in a folder with a specific suffix (txt). You will need the Python os package, see https://docs.python.org/3/library/os.html. You will return the file names in a list.

Use function:

In [236]:
import os

def get_files(fileDir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param filedir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(fileDir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [237]:
files = get_files("selma", ".txt")

In [238]:
files

['troll.txt',
 'kejsaren.txt',
 'marbacka.txt',
 'herrgard.txt',
 'nils.txt',
 'osynliga.txt',
 'jerusalem.txt',
 'bannlyst.txt',
 'gosta.txt']

## Creating a master index

Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions. (a posting list)

In [241]:

def addAll(fileDir,files): 
    """
    Reads all files in list and matches to txt files
    :param dict:
    :param dir:
    :param files:
    :return dict:
    """
    allDict = {}

    for file in files:
        text = open(fileDir+"/"+file).read()  
        txt = txtClean(text)
        stringList = re.findall(r"[a-zåäö]+",text)
        allDict.update(dict.fromkeys(stringList))
    
    for word in allDict: # Iterate through every word in dict
        wordIndices = []
        pattern = r"\b"+word+ r"\b" # Only look at word
        fileDict = dict.fromkeys(files)
        
        for file in files: # Iterate through every text
            text = open(fileDir+"/"+file).read()  
            for m in re.finditer(pattern, text): #Iterate through every word in text
                wordIndices.append(m.start(0))
            fileDict.update({file:wordIndices})
            
        allDict.update({word:fileDict})
    
    return retDict

In [ ]:
masterDict = addAll('selma',files)

In [ ]:
with open('masterDict.pickle', 'wb') as handle:
    pickle.dump(masterDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Below is an except of the master index with the words samlar and ände:

'samlar': {'nils.txt': [53499, 120336], 'gosta.txt': [317119, 414300, 543686],'osynliga.txt': [410995, 871322]},

'ände':{'nils.txt': [3991],'kejsaren.txt': [51100],'marbacka.txt': [374231],'troll.txt': [39726],'osynliga.txt': [742747]},

The word samlar, for instance, occurs three times in the gosta text at positions 317119, 414300, and 543686.

In [ ]:
masterDict["samlar"]

In [ ]:
masterDict["ände"]